In [ ]:
using JuMP
using CPLEX
Pkg.add("OffsetArrays") # -> Nos permitirá usar matrices que empiecen con índice 0 
using OffsetArrays

# 0-1 Knapsack

El problema de knapsack es posiblemente uno de los modelos clásicos más simples que hay. Ofrece una formulación de programación entera simple que además ha sido muy tratada en la literatura. Antes de nada veamos un modelo (sacado de la documentación de JuMP https://github.com/JuliaOpt/JuMP.jl/blob/master/examples/knapsack.jl)

Trataremos la versión 0-1 que es la que aparece en el modelo y luego veremos los cambios necesarios para tratar la versión general (que usamos en el ejemplo de column generation)

In [ ]:
m = Model(solver=CplexSolver())

@variable(m, xVariables[1:5], Bin)

profit = [ 5, 3, 2, 7, 4 ]
weight = [ 2, 8, 4, 2, 5 ]
capacity = 10

# Objective: maximize profit
@objective(m, Max, dot(profit, xVariables))

# Constraint: can carry all
@constraint(m, dot(weight, xVariables) <= capacity)

# Solve problem using MIP solver
status = solve(m)

println("Objective is: ", getobjectivevalue(m))
println("Solution is:")
for i = 1:5
    print("x[$i] = ", getvalue(xVariables[i]),"\n")
end

## 0-1 Knapsack. Método 1. Tabla completa

Cada etapa (columna de la tabla) corresponde a un item. Los estados representan la disponibilidad.

No se muestra cómo reconstruir los valores porque es algo tedioso y la verdad no muy útil como se verá en la siguiente implementación

In [ ]:
function knapsack_m1(p,w,c)
    T=size(p,1)
    tableProfit = OffsetArray(Int64, 1:T, 0:c)
    for t in 1:T
        for s in 0:c
            tableProfit[t,s]=0
        end
    end
    #etapa 1
    tableProfit[1,0]=0
    tableProfit[1,w[1]]=p[1]
    #etapa 2 to T
    for t in 2:T
        s=c-w[t]
        while s>=0
            if tableProfit[t-1,s]>0 
                if tableProfit[t,s] < tableProfit[t-1,s]
                    tableProfit[t,s] = tableProfit[t-1,s]
                end
                if tableProfit[t,s+w[t]]<(tableProfit[t-1,s]+p[t])
                    tableProfit[t,s+w[t]]=tableProfit[t-1,s]+p[t]
                end
            end
            s -= 1
        end
    end
    beneficio=0
    maxLoad=0
    previous=0
    for s in 0:c
        if beneficio < tableProfit[T,s]
            beneficio=tableProfit[T,s]
        end
    end
    return beneficio
end

In [ ]:
beneficio = knapsack_m1(profit,weight,capacity)
println(beneficio)

## 0-1 Knapsack. Método 2. Tabla reducida

En realidad no es necesario tener en cuenta cada etapa por separado. Veremos que es suficiente con vectores si vamos con cuidado en el orden de las operaciones

In [ ]:
function knapsack_m2(p,w,c)
    T=size(p,1)
    items=collect(1:T)
    sort!(items,by=a->(p[a]/w[a]),rev=true) #https://rosettacode.org/wiki/Sort_an_array_of_composite_structures#Julia    
    Profit = OffsetArray(Int64, 0:c)
    Path = OffsetArray(Int64, 0:c)
    Profit[0]=0
    Path[0]=0
    for i in 1:c
        Profit[i]= -1
    end
    for i in 1:c
        Path[i]= -1
    end
    for t in 1:T
        i=items[t]
        s=c-w[i]
        while s>=0
            #first check if current assignment is feasible
            if Path[s]>=0 #first check if current assignment is feasible
                if Profit[s+w[i]]< (Profit[s]+p[i])
                    Profit[s+w[i]]=Profit[s]+p[i]
                    Path[s+w[i]]=i
                end
            end
            s -= 1
        end
        #println(t,"\tusing ",i,"\t",Profit,"\t\t",Path)
    end
    ySol=zeros(Int64,T)
    #first find position, then backtrack
    ind=indmax(Profit)-1 #rectificar el -1 porque el vector va de 0 a c, no de 1 a c
    #println("index: ",ind,"\t",Profit[ind])
    while ind!=0
        ySol[Path[ind]]=1
        ind -= w[Path[ind]]
    end
    return maximum(Profit),ySol
end

In [ ]:
beneficio, ySol = knapsack_m2(profit,weight,capacity)
println(beneficio,"\t",ySol)

### Pregunta

¿Por qué ha sido necesario reordenar los items?

## integer Knapsack. Programación entera

Sólo debemos cambiar del programa entero la declaración de las variables. Suponiendo que tenemos el modelo anterior cargado y ejecutado...

In [ ]:
for i=1:length(xVariables)
    setcategory(xVariables[i], :Int)
    setlowerbound(xVariables[i],0.0)
    setupperbound(xVariables[i],10000.0)
end
status = solve(m)
println("Objective is: ", getobjectivevalue(m))
println("Objective is: ", getobjectivevalue(model))
println("Solution is:")
for i = 1:5
    print("x[$i] = ", getvalue(xVariables[i]),"\n")
end

## Integer Knapsack. Tabla reducida

Ahora el método cambiará en el orden de exploración de los estados

In [ ]:
function knapsackInt(p,w,c)
    T=size(p,1)
    Profit = OffsetArray(Int64, 0:c)
    Path = OffsetArray(Int64, 0:c)
    Profit[0]=0
    Path[0]=0
    for i in 1:c
        Profit[i]= -1
    end
    for i in 1:c
        Path[i]= -1
    end
    for t in 1:T
        s=0
        while s<=(c-w[t])
            #first check if current assignment is feasible
            if Path[s]>=0 #first check if current assignment is feasible
                if Profit[s+w[t]]< (Profit[s]+p[t])
                    Profit[s+w[t]]=Profit[s]+p[t]
                    Path[s+w[t]]=t
                end
            end
            s += w[t]
        end
        println(t,"\t",Profit,"\t\t",Path)
    end
    ySol=zeros(Int64,T)
    #first find position, then backtrack
    ind=indmax(Profit)-1 #rectificar el -1 porque el vector va de 0 a c, no de 1 a c
    #println("index: ",ind,"\t",Profit[ind])
    while ind!=0
        ySol[Path[ind]] += 1
        ind -= w[Path[ind]]
    end
    return maximum(Profit),ySol
end

In [ ]:
beneficio, ySol = knapsackInt(profit,weight,capacity)
println(beneficio,"\t",ySol)

### Pregunta

¿Por qué aquí el procedimiento no funcionaría si la exploración de estados se hubiera ordenado como en el knapsack 0-1?

## Ejercicio

Integrar la función knapsackInt con el problema de generación de columnas